# Read input data

In [ ]:
import re
import pprint
import os
# import numpy as np
# from scipy.stats import chi2

# Define a pattern to match the relevant data
RE_IGNORE_OPTIONAL_LINE = r'(?:[^\n]*)?'
pattern = [
    r'Running ([\./\w]+) on (\w+)[^\n]*',
    RE_IGNORE_OPTIONAL_LINE,
    RE_IGNORE_OPTIONAL_LINE,
    r'Number of threads: (\d+)',
    r'Runtime: (\d+\.[\d]+)'
]
pattern = r'\n?'.join(pattern) # This ? may cause unexpected behavior
print(f'Pattern: {pattern}')

pattern_beamer = [
    r'Running ([\./\w]+) on (\w+)[^\n]*',
    RE_IGNORE_OPTIONAL_LINE,
    RE_IGNORE_OPTIONAL_LINE,
    RE_IGNORE_OPTIONAL_LINE,
    RE_IGNORE_OPTIONAL_LINE,
    r'Average Time: +(\d+\.[\d]+)'
]
pattern_beamer = r'\n?'.join(pattern_beamer) # This ? may cause unexpected behavior
print(f'Pattern reference: {pattern_beamer}')

# List all files in the 'data_runtime' directory
all_files = os.listdir('data_runtime')

# Filter files that start with 'runtime_'
runtime_files = [f for f in all_files if f.startswith('runtime_')]

runtimes = {}

for runtime_file in runtime_files:
    print(f'Parsing {runtime_file}...')
    with open(f'data_runtime/{runtime_file}', 'r') as file:
        file_format = file.read()

    # Find all matches in the string
    matches = re.findall(pattern if 'beamer' not in runtime_file else pattern_beamer, file_format)
    print(f'Found {len(matches)} matches')
    program = runtime_file[len('runtime_'):-4]
    program = ''.join([i for i in program if not i.isdigit()])
    
    # Convert matches to a list of dictionaries
    for match in matches:
        if 'beamer' in runtime_file:
            binary = match[0]
            dataset = match[1]
            threads = 32
            runtime = float(match[2])
        else:
            binary = match[0]
            dataset = match[1]
            threads = int(match[2])
            runtime = float(match[3])

        # Init dictionary
        if runtimes.get(program) == None:
            runtimes[program] = {}
        if runtimes[program].get(threads) == None:
            runtimes[program][threads] = {}
        if runtimes[program][threads].get(dataset) == None:
            runtimes[program][threads][dataset] = []

        runtimes[program][threads][dataset].append({'binary': binary, 'threads': threads, 'runtime': runtime})

# pprint.pprint(runtimes)

# for program, threads_l in runtimes.items():
#     for threads, dataset in threads_l.items():
#         for dataset, entries in dataset.items():
#             runtime_values = [float(entry['runtime']) for entry in entries]
#             # print(f'{program} on {dataset}: {runtime_values}')
            
#             # Expected variance (threshold for acceptable noise)
#             expected_variance = 0.1

#             # Sample variance
#             sample_variance = np.var(runtime_values, ddof=1)

#             # Number of samples
#             n = len(runtime_values)

#             # Chi-square statistic
#             chi2_stat = (n - 1) * sample_variance / expected_variance

#             # p-value (two-tailed test)
#             p_value = 2 * min(chi2.cdf(chi2_stat, n - 1), 1 - chi2.cdf(chi2_stat, n - 1))

#             # print(f"Sample Variance: {sample_variance}")
#             # print(f"Chi-Square Statistic: {chi2_stat}")
#             # print(f"P-Value: {p_value}")

#             # Interpretation
#             alpha = 0.05  # Significance level
#             if p_value < alpha:
#                 print("The variance is significantly different from the expected value.")
#             else:
#                 print("The variance is NOT significantly different from the expected value.")


average_runtimes = {}
for program, threads_l in runtimes.items():
    if average_runtimes.get(program) is None:
        average_runtimes[program] = {}
    for threads, datasets in threads_l.items():
        if average_runtimes[program].get(threads) is None:
            average_runtimes[program][threads] = {}
        for dataset, entries in datasets.items():
            total_runtime = sum(float(entry['runtime']) for entry in entries)
            average_runtime = total_runtime / len(entries)
            average_runtimes[program][threads][dataset] = average_runtime

sequential_naive = None
if 'sequential_naive' in average_runtimes:
    sequential_naive = average_runtimes['sequential_naive'][list(average_runtimes['sequential_naive'].keys())[0]]
    del average_runtimes['sequential_naive']

pprint.pprint(average_runtimes)

# Generate plots on runtime and speedups

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

FONT_TITLE = 18
FONT_AXES = 18
FONT_TICKS = 14
FONT_LEGEND = 14
dataset_sizes = {
    'Social_Network_1':         (4799030,83971060),
    'Web_Graph_1':              (6558851,294259270),
    'Collaboration_Network_1':  (1058365,110391514),
    'Synthetic_Dense_1':        (9899865,980075384),
    'Road_Network_1':           (21872120,58216498),
    'kNN_Graph_1':              (24627858,154302284),
    'Synthetic_Sparse_1':       (9899865,39202750),
    'Road_Network_2':           (86081964,215900378),
}
def human_readable_size(size):
    if size >= 1e9:
        return f'{size / 1e9:.1f} B'
    else:
        return f'{size / 1e6:.1f} M'
for dataset, (nodes, edges) in dataset_sizes.items():
    print(f'{dataset:<30}: {human_readable_size(nodes):<5} nodes, {human_readable_size(edges)} edges')

dataset_category = {
    'Social_Network_1':         'small',
    'Web_Graph_1':              'small',
    'Collaboration_Network_1':  'small',
    'Synthetic_Dense_1':        'small',
    'Road_Network_1':           'large',
    'kNN_Graph_1':              'large',
    'Synthetic_Sparse_1':       'large',
    'Road_Network_2':           'large',
}
category_colors = {
    'small': 'blue',
    'large': 'green',
    'verylarge': 'green',
}
datasets = list(dataset_category.keys())
BASELINE_PROGRAM = 'sbeamer_four_twfour' # sbeamer_four_twfour sbeamer_fiftn_eigtn
programs = [
    # 'small96',
    # 'large96',
    # 'verylarge96',
    # 'auto96',

    # 'small24',
    # 'large24',
    # 'verylarge24',
    # 'auto24',

    BASELINE_PROGRAM,
    'small',
    'large',
]

PROGRAM_LABELS = {
    'small': 'Bitmap',
    'large': 'MergedCSR',
}

# Calculate speedups
speedups = {}
for program in programs:
    program_threads = average_runtimes.get(program, {})
    speedups[program] = {}
    for threads, program_datasets in program_threads.items():
        speedups[program][threads] = {}
        for dataset, runtime in program_datasets.items():
            if threads in average_runtimes[BASELINE_PROGRAM]:
                if dataset in average_runtimes[BASELINE_PROGRAM][threads]:
                    speedups[program][threads][dataset] = average_runtimes[BASELINE_PROGRAM][threads][dataset] / runtime
            else:
                speedups[program][threads][dataset] = 0

eps = {}
for program in programs:
    program_threads = average_runtimes.get(program, {})
    eps[program] = {}
    for threads, program_datasets in program_threads.items():
        eps[program][threads] = {}
        for dataset, runtime in program_datasets.items():
            if threads in average_runtimes[program]:
                if dataset in average_runtimes[program][threads]:
                    eps[program][threads][dataset] = dataset_sizes[dataset][1] / runtime
            else:
                eps[program][threads][dataset] = 0

if sequential_naive is not None:
    speedups_naive = {}
    for program in programs:
        program_threads = average_runtimes.get(program, {})
        speedups_naive[program] = {}
        for threads, program_datasets in program_threads.items():
            speedups_naive[program][threads] = {}
            for dataset, runtime in program_datasets.items():
                if dataset in average_runtimes[program][threads]:
                    speedups_naive[program][threads][dataset] = sequential_naive[dataset] / runtime

speedups_scalability = {}
for program in programs:
    if program == BASELINE_PROGRAM:
        continue
    program_threads = average_runtimes.get(program, {})
    speedups_scalability[program] = {}
    for threads, program_datasets in program_threads.items():
        speedups_scalability[program][threads] = {}
        for dataset, runtime in program_datasets.items():
            ref_runtime = average_runtimes[program][1].get(dataset)
            # if ref_runtime is None:
            #     print(f'WARNING: Reference runtime for {program} on {dataset} with 1 thread is None and has been set to 60s')
            speedups_scalability[program][threads][dataset] = (60 if ref_runtime is None else ref_runtime) / runtime

print("===== SPEEDUPS =====")
pprint.pprint(speedups)
print("\n===== SPEEDUPS SCALABILITY =====")
pprint.pprint(speedups_scalability)
print("\n===== Edges/s =====")
pprint.pprint(eps)

## Plot speedups

In [ ]:
small_datasets = [d for d in datasets if dataset_category[d] == 'small']
large_datasets = [d for d in datasets if dataset_category[d] == 'large']

FONT_TITLE = 18
FONT_AXES = 18
FONT_LEGEND = 14
BAR_WIDTH = 0.3

# _threads = sorted(average_runtimes[BASELINE_PROGRAM].keys())
_threads = [32]

for threads in _threads:
    max_eps = -1
    # Plot speedups for small datasets
    fig, ax = plt.subplots(figsize=(5, 5))
    for i, (program, program_speedups) in enumerate(speedups.items()):
        if program != BASELINE_PROGRAM:
            small_datasets = [d for d in datasets if dataset_category[d] == 'small']
            program_speedups = program_speedups[threads]
            small_speedups = [program_speedups.get(dataset, np.nan) for dataset in small_datasets]
            bars = ax.bar(np.arange(len(small_datasets)) + i * BAR_WIDTH, small_speedups, BAR_WIDTH, label=PROGRAM_LABELS.get(program, program))
            for bar, speedup in zip(bars, small_speedups):
                if np.isnan(speedup):
                    ax.text(bar.get_x() + bar.get_width() / 4, 0, 'x')
                else:
                    max_eps = max(max_eps, speedup)
    max_eps = int(max_eps) + 2

    # Add some text for labels, title and custom x-axis tick labels, etc.
    # ax.set_xlabel('Small Datasets')
    # ax.set_ylabel('Speedup', fontsize=FONT_AXES)
    speedup_ticks = np.arange(max_eps, step=0.25)
    ax.plot(range(5), [1] * int(5), color='black', linestyle=':')
    ax.set_title(f'Speedups ({threads} threads)', fontsize=FONT_TITLE)
    ax.set_yticks(speedup_ticks)
    ax.set_yticklabels([f'{i}x' for i in speedup_ticks], fontsize=FONT_TICKS)
    ax.set_xticks(np.arange(len(small_datasets)) + 1.5 * BAR_WIDTH)
    ax.set_xticklabels(small_datasets, rotation=45, ha='right', fontsize=FONT_TICKS-2)
    # for tick, dataset in zip(ax.get_xticklabels(), small_datasets):
    #     tick.set_color(category_colors[dataset_category[dataset]])
    ax.legend(loc='upper left', fontsize=FONT_LEGEND)
    ax.grid(True, linestyle=':')
    fig.tight_layout()
    plt.show()

    max_eps = -1
    # Plot speedups for large datasets
    fig, ax = plt.subplots(figsize=(5, 5))
    for i, (program, program_speedups) in enumerate(speedups.items()):
        if program != BASELINE_PROGRAM:
            large_datasets = [d for d in datasets if dataset_category[d] == 'large']
            program_speedups = program_speedups[threads]
            large_speedups = [program_speedups.get(dataset, np.nan) for dataset in large_datasets]
            bars = ax.bar(np.arange(len(large_datasets)) + i * BAR_WIDTH, large_speedups, BAR_WIDTH, label=PROGRAM_LABELS.get(program, program))
            for bar, speedup in zip(bars, large_speedups):
                if np.isnan(speedup):
                    ax.text(bar.get_x() + bar.get_width() / 4, 0, 'x')
                else:
                    max_eps = max(max_eps, speedup)
    max_eps = int(max_eps) + 2

    # Add some text for labels, title and custom x-axis tick labels, etc.
    # ax.set_xlabel('Large Datasets')
    # ax.set_ylabel('Speedup', fontsize=FONT_AXES)
    speedup_ticks = np.arange(max_eps, step=0.25)
    ax.plot(range(5), [1] * int(5), color='black', linestyle=':')
    ax.set_title(f'Speedups ({threads} threads)', fontsize=FONT_TITLE)
    ax.set_yticks(speedup_ticks)
    ax.set_yticklabels([f'{i:.2f}x' for i in speedup_ticks], fontsize=FONT_TICKS)
    ax.set_xticks(np.arange(len(large_datasets)) + 1.5 * BAR_WIDTH)
    ax.set_xticklabels(large_datasets, rotation=45, ha='right', fontsize=FONT_TICKS-2)
    # for tick, dataset in zip(ax.get_xticklabels(), large_datasets):
    #     tick.set_color(category_colors[dataset_category[dataset]])
    ax.legend(loc='upper left', fontsize=FONT_LEGEND)
    ax.grid(True, linestyle=':')
    fig.tight_layout()
    plt.show()

## Plot Edges/s

In [ ]:
import pandas as pd
_threads = [32]

for threads in _threads:
    # Create a DataFrame for eps
    eps_data = []
    for program, threads_data in eps.items():
        threads_data = threads_data[threads]
        for dataset, value in threads_data.items():
            eps_data.append([program, threads, dataset, value])

    eps_df = pd.DataFrame(eps_data, columns=['Program', 'Threads', 'Dataset', 'Edges/s'])
    eps_df_pivot = eps_df.pivot_table(index=['Dataset', 'Threads'], columns='Program', values='Edges/s')
    # print(eps_df_pivot)

    def format_number(x):
        if x >= 1e9:
            return f'{x / 1e9:.1f}B'
        elif x >= 1e6:
            return f'{x / 1e6:.1f}M'
        else:
            return f'{x:.1f}'
    formatted_eps_df_pivot = eps_df_pivot.map(format_number)
    print(formatted_eps_df_pivot)
    print(formatted_eps_df_pivot.to_latex())

## Plot speedups wrt to naive serial

In [ ]:
# _threads = sorted(average_runtimes[BASELINE_PROGRAM].keys())
_threads = [32]

if sequential_naive is not None and speedups_naive is not None:
    for threads in _threads:
        max_eps = -1
        # Plot speedups for small datasets
        fig, ax = plt.subplots(figsize=(5, 5))
        for i, (program, program_speedups) in enumerate(speedups_naive.items()):
            if program != BASELINE_PROGRAM:
                small_datasets = [d for d in datasets if dataset_category[d] == 'small']
                program_speedups = program_speedups[threads]
                small_speedups = [program_speedups.get(dataset, np.nan) for dataset in small_datasets]
                bars = ax.bar(np.arange(len(small_datasets)) + i * BAR_WIDTH, small_speedups, BAR_WIDTH, label=program)
                for bar, speedup in zip(bars, small_speedups):
                    if np.isnan(speedup):
                        ax.text(bar.get_x() + bar.get_width() / 4, 0, 'x')
                    else:
                        max_eps = max(max_eps, speedup)
        max_eps = int(max_eps) + 2

        # Add some text for labels, title and custom x-axis tick labels, etc.
        # ax.set_xlabel('Small Datasets')
        # ax.set_ylabel('Speedup', fontsize=FONT_AXES)
        ax.set_title(f'Speedups Using {threads} Threads', fontsize=FONT_TITLE)
        # ax.set_yticks(np.arange(max_speedup))
        # ax.set_yticklabels([f'{i}x' for i in range(max_speedup)], fontsize=FONT_TICKS)
        ax.set_xticks(np.arange(len(small_datasets)) + 1.5 * BAR_WIDTH)
        ax.set_xticklabels(small_datasets, rotation=45, ha='right', fontsize=FONT_TICKS-2)
        # for tick, dataset in zip(ax.get_xticklabels(), small_datasets):
        #     tick.set_color(category_colors[dataset_category[dataset]])
        ax.legend(loc='upper left', fontsize=FONT_LEGEND)
        ax.grid(True, linestyle=':')
        fig.tight_layout()
        plt.show()

        max_eps = -1
        # Plot speedups for large datasets
        fig, ax = plt.subplots(figsize=(5, 5))
        for i, (program, program_speedups) in enumerate(speedups_naive.items()):
            if program != BASELINE_PROGRAM:
                large_datasets = [d for d in datasets if dataset_category[d] == 'large']
                program_speedups = program_speedups[threads]
                large_speedups = [program_speedups.get(dataset, np.nan) for dataset in large_datasets]
                bars = ax.bar(np.arange(len(large_datasets)) + i * BAR_WIDTH, large_speedups, BAR_WIDTH, label=program)
                for bar, speedup in zip(bars, large_speedups):
                    if np.isnan(speedup):
                        ax.text(bar.get_x() + bar.get_width() / 4, 0, 'x')
                    else:
                        max_eps = max(max_eps, speedup)
        max_eps = int(max_eps) + 2

        # Add some text for labels, title and custom x-axis tick labels, etc.
        # ax.set_xlabel('Large Datasets')
        # ax.set_ylabel('Speedup', fontsize=FONT_AXES)
        ax.set_title(f'Speedups Using {threads} Threads', fontsize=FONT_TITLE)
        # ax.set_yticks(np.arange(max_speedup))
        # ax.set_yticklabels([f'{i}x' for i in range(max_speedup)], fontsize=FONT_TICKS)
        ax.set_xticks(np.arange(len(large_datasets)) + 1.5 * BAR_WIDTH)
        ax.set_xticklabels(large_datasets, rotation=45, ha='right', fontsize=FONT_TICKS-2)
        # for tick, dataset in zip(ax.get_xticklabels(), large_datasets):
        #     tick.set_color(category_colors[dataset_category[dataset]])
        ax.legend(loc='upper left', fontsize=FONT_LEGEND)
        ax.grid(True, linestyle=':')
        fig.tight_layout()
        plt.show()

In [ ]:
from scipy.stats import gmean

small_datasets = [d for d in datasets if dataset_category[d] == 'small']
large_datasets = [d for d in datasets if dataset_category[d] == 'large']

geometric_means = {}
for program, threads_data in eps.items():
    geometric_means[program] = {'small': {}, 'large': {}}
    for threads, datasets_data in threads_data.items():
        small_values = [value for dataset, value in datasets_data.items() if dataset in small_datasets and value > 0]
        large_values = [value for dataset, value in datasets_data.items() if dataset in large_datasets and value > 0]
        
        if small_values:
            geometric_means[program]['small'][threads] = gmean(small_values)
        else:
            geometric_means[program]['small'][threads] = np.nan
        
        if large_values:
            geometric_means[program]['large'][threads] = gmean(large_values)
        else:
            geometric_means[program]['large'][threads] = np.nan

pprint.pprint(geometric_means)

In [ ]:
for threads in [32]:
    print(f'SMALL {threads} threads')
    print(f'\t{"Beamer":<10} {human_readable_size(geometric_means["sbeamer_four_twfour"]["small"][threads])}edges/s')
    print(f'\t{"Small":<10} {human_readable_size(geometric_means["small"]["small"][threads])}edges/s')
    print(f'\t{"Ratio":<10} {geometric_means["small"]["small"][threads] / geometric_means["sbeamer_four_twfour"]["small"][threads]:.2f}')
    print(f'LARGE {threads} threads')
    print(f'\t{"Beamer":<10} {human_readable_size(geometric_means["sbeamer_four_twfour"]["large"][threads])}edges/s')
    print(f'\t{"Large":<10} {human_readable_size(geometric_means["large"]["large"][threads])}edges/s')
    print(f'\t{"Ratio":<10} {geometric_means["large"]["large"][threads] / geometric_means["sbeamer_four_twfour"]["large"][threads]:.2f}')

# Generate scalability plots

In [ ]:
FONT_TITLE = 18
FONT_AXES = 18
FONT_TICKS = 14
FONT_LEGEND = 14

for program in programs:
    if program == BASELINE_PROGRAM:
        continue
    fig, ax = plt.subplots(figsize=(10, 5))
    thread_counts = sorted(speedups_scalability[program].keys())
    for dataset in datasets:
        if program == dataset_category[dataset]:
            speedup_values = [speedups_scalability[program][threads].get(dataset, np.nan) for threads in thread_counts]
            ax.plot(thread_counts, speedup_values, marker='o', label=dataset)
    
    # Ideal speedup
    ax.plot(thread_counts, thread_counts, '--', color='gray', label='Ideal Speedup')

    ax.set_xlabel('Number of Threads', fontsize=FONT_AXES)
    ax.set_ylabel('Speedup (w.r.t. 1 thread)', fontsize=FONT_AXES)
    ax.set_title(f'Scaling on graphs with {"large" if program=="small" else "small"} frontiers', fontsize=FONT_TITLE)
    ax.set_xticks(thread_counts)
    ax.set_xticklabels(thread_counts, fontsize=FONT_TICKS)
    ax.set_yticks(thread_counts)
    ax.set_yticklabels(thread_counts, fontsize=FONT_TICKS)
    # ax.set_xlim((0,8))
    ax.set_ylim((0,32))
    ax.legend(loc='best', fontsize=FONT_LEGEND)
    ax.grid(True, linestyle=':')
    plt.show()